## Preprocessing the Data

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("Data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


# -------------------------------------------------------

## Optimization Attempt #1 - Remove Columns

In [3]:
# Drop columns
removed_col_df = application_df.drop(columns=['EIN',
                             'NAME',
                             'APPLICATION_TYPE',
                             'CLASSIFICATION'
                            ])

In [4]:
# Determine the number of unique values in each column.
removed_col_df.nunique()

AFFILIATION                  6
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [5]:
# Generate our categorical variable lists
removed_col_cat = removed_col_df.dtypes[
    removed_col_df.dtypes=='object'].index.tolist()

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(
    removed_col_df[removed_col_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(removed_col_cat)
encode_df.head()

,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [7]:
# Merge one-hot encoded features and drop the originals
removed_col_df = removed_col_df.merge(encode_df, left_index=True,
                                     right_index=True)
removed_col_df = removed_col_df.drop(removed_col_cat,axis=1)
removed_col_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,AFFILIATION_CompanySponsored,AFFILIATION_Family/Parent,AFFILIATION_Independent,AFFILIATION_National,AFFILIATION_Other,AFFILIATION_Regional,USE_CASE_CommunityServ,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
removed_col_df.shape

(34299, 29)

In [9]:
# Split our preprocessed data into our features and target arrays
y = removed_col_df['IS_SUCCESSFUL'].values
X = removed_col_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [11]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 15
hidden_nodes_layer2 = 10

nn1 = tf.keras.models.Sequential()

# First hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features,
                             activation='relu'))

# Second hidden layer
nn1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, 
                             activation='sigmoid'))

# Check the structure of the model
nn1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 15)                435       
                                                                 
 dense_1 (Dense)             (None, 10)                160       
                                                                 
 dense_2 (Dense)             (None, 1)                 11        
                                                                 
Total params: 606
Trainable params: 606
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Compile the model
nn1.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

In [13]:
# Train the model
fit_model = nn1.fit(X_train_scaled,y_train,
                   epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.6200 - accuracy: 0.6754
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.6010 - accuracy: 0.6983
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.6001 - accuracy: 0.6996
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5990 - accuracy: 0.6988
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5992 - accuracy: 0.7000
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5991 - accuracy: 0.7000
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5987 - accuracy: 0.7004
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5984 - accuracy: 0.7006
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5984 - accuracy: 0.7013
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5981 - accuracy: 0.7012

In [14]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6048 - accuracy: 0.6949 - 454ms/epoch - 2ms/step
Loss: 0.6048365831375122, Accuracy: 0.6949270963668823


# -------------------------------------------------------

## Optimization Attempt #2 - Increase Number of Neuron Units

In [15]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [16]:
apptype_counts = application_df['APPLICATION_TYPE'].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(apptype_counts[apptype_counts < 500].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [17]:
class_counts = application_df['CLASSIFICATION'].value_counts()

# Determine which values to replace if counts are less than ..?
replace_class = list(class_counts[class_counts < 1000].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [18]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[
    application_df.dtypes=='object'].index.tolist()

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(
    application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)
encode_df.head()

,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,AFFILIATION_CompanySponsored,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [19]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,
                                     right_index=True)
application_df = application_df.drop(application_cat,axis=1)
application_df.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,1,108590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,1,5000,0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,6692,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,142590,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [20]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100

nn2 = tf.keras.models.Sequential()

# First hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features,
                             activation='relu'))

# Second hidden layer
nn2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, 
                             activation='sigmoid'))

# Check the structure of the model
nn2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 150)               6600      
                                                                 
 dense_4 (Dense)             (None, 100)               15100     
                                                                 
 dense_5 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21,801
Trainable params: 21,801
Non-trainable params: 0
_________________________________________________________________


In [23]:
# Compile the model
nn2.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

In [24]:
# Train the model
fit_model = nn2.fit(X_train_scaled,y_train,
                   epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5698 - accuracy: 0.7247
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7299
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5520 - accuracy: 0.7308
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5502 - accuracy: 0.7335
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7314
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5473 - accuracy: 0.7331
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5456 - accuracy: 0.7350
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5458 - accuracy: 0.7344
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5446 - accuracy: 0.7363
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5443 - accuracy: 0.7364

In [25]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5635 - accuracy: 0.7242 - 450ms/epoch - 2ms/step
Loss: 0.5635349750518799, Accuracy: 0.7241982221603394


# -------------------------------------------------------

## Optimization Attempt #3 - Increase Number of Epochs

In [26]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100

nn3 = tf.keras.models.Sequential()

# First hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features,
                             activation='relu'))

# Second hidden layer
nn3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, 
                             activation='sigmoid'))

# Check the structure of the model
nn3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 150)               6600      
                                                                 
 dense_7 (Dense)             (None, 100)               15100     
                                                                 
 dense_8 (Dense)             (None, 1)                 101       
                                                                 
Total params: 21,801
Trainable params: 21,801
Non-trainable params: 0
_________________________________________________________________


In [27]:
# Compile the model
nn3.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

In [28]:
# Train the model
fit_model = nn3.fit(X_train_scaled,y_train,
                   epochs=70)

Epoch 1/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5688 - accuracy: 0.7226
Epoch 2/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5553 - accuracy: 0.7280
Epoch 3/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5508 - accuracy: 0.7313
Epoch 4/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5493 - accuracy: 0.7314
Epoch 5/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5483 - accuracy: 0.7323
Epoch 6/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5474 - accuracy: 0.7343
Epoch 7/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5462 - accuracy: 0.7345
Epoch 8/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5455 - accuracy: 0.7337
Epoch 9/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5444 - accuracy: 0.7354
Epoch 10/70
804/804 [==============================] - 1s 2ms/step - loss: 0.5442 - accuracy: 0.7357

In [29]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.6235 - accuracy: 0.7236 - 437ms/epoch - 2ms/step
Loss: 0.6234772801399231, Accuracy: 0.7236151695251465


# -------------------------------------------------------

## Optimization Attempt #4 - Change Activation Function

In [30]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100

nn4 = tf.keras.models.Sequential()

# First hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features,
                             activation='tanh'))

# Second hidden layer
nn4.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation='tanh'))

# Output layer
nn4.add(tf.keras.layers.Dense(units=1, 
                             activation='sigmoid'))

# Check the structure of the model
nn4.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_9 (Dense)             (None, 150)               6600      
                                                                 
 dense_10 (Dense)            (None, 100)               15100     
                                                                 
 dense_11 (Dense)            (None, 1)                 101       
                                                                 
Total params: 21,801
Trainable params: 21,801
Non-trainable params: 0
_________________________________________________________________


In [31]:
# Compile the model
nn4.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

In [32]:
# Train the model
fit_model = nn4.fit(X_train_scaled,y_train,
                   epochs=20)

Epoch 1/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5749 - accuracy: 0.7208
Epoch 2/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5591 - accuracy: 0.7281
Epoch 3/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5534 - accuracy: 0.7297
Epoch 4/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5518 - accuracy: 0.7296
Epoch 5/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5495 - accuracy: 0.7318
Epoch 6/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5482 - accuracy: 0.7327
Epoch 7/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5470 - accuracy: 0.7345
Epoch 8/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5465 - accuracy: 0.7349
Epoch 9/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5453 - accuracy: 0.7334
Epoch 10/20
804/804 [==============================] - 1s 2ms/step - loss: 0.5443 - accuracy: 0.7355

In [33]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn4.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5595 - accuracy: 0.7238 - 485ms/epoch - 2ms/step
Loss: 0.5594739317893982, Accuracy: 0.7238484025001526


# -------------------------------------------------------

## Optimization Attempt #5 - Add Hidden Layers and Increase Number of Epochs

In [34]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 150
hidden_nodes_layer2 = 100
hidden_nodes_layer3 = 100
hidden_nodes_layer4 = 100

nn5 = tf.keras.models.Sequential()

# First hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                             input_dim=number_input_features,
                             activation='relu'))

# Second hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, 
                             activation='relu'))

# Third hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, 
                             activation='relu'))

# Fourth hidden layer
nn5.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, 
                             activation='relu'))
# Output layer
nn5.add(tf.keras.layers.Dense(units=1, 
                             activation='sigmoid'))

# Check the structure of the model
nn5.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 150)               6600      
                                                                 
 dense_13 (Dense)            (None, 100)               15100     
                                                                 
 dense_14 (Dense)            (None, 100)               10100     
                                                                 
 dense_15 (Dense)            (None, 100)               10100     
                                                                 
 dense_16 (Dense)            (None, 1)                 101       
                                                                 
Total params: 42,001
Trainable params: 42,001
Non-trainable params: 0
_________________________________________________________________


In [35]:
# Compile the model
nn5.compile(loss='binary_crossentropy',
           optimizer='adam',
           metrics=['accuracy'])

In [36]:
# Train the model
fit_model = nn5.fit(X_train_scaled,y_train,
                   epochs=70)

Epoch 1/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5673 - accuracy: 0.7222
Epoch 2/70
804/804 [==============================] - 2s 3ms/step - loss: 0.5543 - accuracy: 0.7308
Epoch 3/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7318
Epoch 4/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5490 - accuracy: 0.7341
Epoch 5/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7339
Epoch 6/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7348
Epoch 7/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5458 - accuracy: 0.7358
Epoch 8/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5450 - accuracy: 0.7363
Epoch 9/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5443 - accuracy: 0.7364
Epoch 10/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5437 - accuracy: 0.7371

In [37]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.8600 - accuracy: 0.7224 - 484ms/epoch - 2ms/step
Loss: 0.8599571585655212, Accuracy: 0.722449004650116


## Save Checkpoints and Results

In [38]:
# define checkpoint path and filenames
os.makedirs("optimization_checkpoints/", exist_ok=True)
checkpoint_path5 = "optimization_checkpoints/weights.{epoch:02d}.hdf5"

# create callback that saves the model's weight coefficients
cp_callback = ModelCheckpoint(filepath=checkpoint_path5,
                              verbose=1,
                              save_weights_only=True,
                              save_freq='epoch',
                              period=5
                             )

# Train the model
fit_model = nn5.fit(X_train_scaled,y_train,
                   epochs=70,
                   callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn5.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

# Export our model to HDF5 file
nn5.save("AlphabetSoupCharity_Optimization.h5")

Epoch 1/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5319 - accuracy: 0.7420
Epoch 2/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5316 - accuracy: 0.7421
Epoch 3/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5333 - accuracy: 0.7420
Epoch 4/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7420
Epoch 5/70
804/804 [==============================] - ETA: 0s - loss: 0.5315 - accuracy: 0.7426
Epoch 5: saving model to optimization_checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5315 - accuracy: 0.7426
Epoch 6/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5322 - accuracy: 0.7420
Epoch 7/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5320 - accuracy: 0.7418
Epoch 8/70
804/804 [==============================] - 2s 2ms/step - loss: 0.5321 - accuracy: 0.7423
Epoch 9/70
804/804 [=============================